In [ ]:
from model.model import *
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
import pickle
from dataset import ModelNet40Dataset
import numpy as np
import math
%matplotlib inline
from IPython.display import clear_output
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
#load data
filehandler = open('ModelNet40ProcessedDataset', 'rb') 
ModelNet40Data = pickle.load(filehandler)

# print(len(ModelNet40Data.train['pcds']))
def sample_data(data,batch_size):
    total_N = len(data['pcds'])
    indices = np.random.choice(total_N,batch_size,replace = False)
    N_pts,d =  data['pcds'][0].shape
    pcds = np.zeros((batch_size,N_pts,d))
    labels = np.zeros(batch_size,dtype=int)
    for i in range(batch_size):
        pcd = data['pcds'][indices[i]]
        #rotate pointcloud
        a = np.random.rand()*math.pi
        R = np.array([[math.cos(a),-math.sin(a),0.],\
              [math.sin(a),math.cos(a),0.],\
              [0.,0.,1.]])
        pcd = pcd.dot(R)
        #jitter
        pcd += np.random.normal(0,0.02,size=pcd.shape)      
        pcds[i,:,:] = pcd
        labels[i] = int(data['labels'][indices[i]])
    return pcds,labels
    
def get_data(data):
    total_N = len(data['pcds'])
    indices = np.arange(total_N)
    N_pts,d =  data['pcds'][0].shape
    pcds = np.zeros((total_N,N_pts,d))
    labels = np.zeros(total_N,dtype=int)
    for i in range(total_N):
        pcds[i,:,:] = data['pcds'][indices[i]]
        labels[i] = int(data['labels'][indices[i]])
    return pcds,labels

def compute_accuracy(preds,target):
    tmp = preds.max(1).indices == target
    return torch.sum(tmp).detach().cpu().numpy()/target.size(0)


In [ ]:
##Hyperparameters
N_CLASSES = 40
EPOCHS = 1000#2000
BATCH_SIZE = 32
INIT_LR = 0.001
MOMENTUM = 0.9
LR_STEP = 20
SCHEDULER_GAMMA = 0.5
TEST_EVERY = 1
REG_WEIGHT = 0.001
criterion = nn.CrossEntropyLoss()

In [ ]:
net = PointNetClassification(N_CLASSES).to(device)
optimizer = optim.Adam(params=net.parameters(), lr=INIT_LR)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=LR_STEP, gamma=SCHEDULER_GAMMA)


In [ ]:
##training loop
train_losses = []
train_iterations = []
test_losses = []
test_iterations = []


for epoch in range(EPOCHS):
    optimizer.zero_grad()
    net.train() 
    #load the batch of data
    data,labels = sample_data(ModelNet40Data.train,BATCH_SIZE)
    data = torch.from_numpy(data).float().to(device)
    labels = torch.from_numpy(labels).to(device)
    
    #compute the loss
    preds,M2 = net(data)
    loss = criterion(preds,labels)
    
    #add transformation matrix regularization loss
    I = torch.eye(64).unsqueeze(0).to(device)
    loss2 = torch.mean(torch.norm(torch.bmm(M2,M2.transpose(2,1)) - I, dim=(1,2)))    
    loss += REG_WEIGHT*loss2
    
    train_losses.append(loss.detach().cpu())
    train_iterations.append(epoch)
    train_accuracy = compute_accuracy(preds,labels)
    #step the optimizer
    
    loss.backward()
    optimizer.step()
    scheduler.step()
    
    if epoch%TEST_EVERY == 0:
        with torch.no_grad():
            net.eval()
            #load the batch of test data (batch size couldn't be too big)
            data,labels = sample_data(ModelNet40Data.test,32)
            data = torch.from_numpy(data).float().to(device)
            labels = torch.from_numpy(labels).to(device)
            
            preds,M2 = net(data)
            test_loss = criterion(preds,labels)
            #add transformation matrix regularization loss
            I = torch.eye(64).unsqueeze(0).to(device)
            test_loss2 = torch.mean(torch.norm(torch.bmm(M2,M2.transpose(2,1)) - I, dim=(1,2)))    
            test_loss += REG_WEIGHT*test_loss2

            test_losses.append(test_loss.detach().cpu())
            test_iterations.append(epoch)
            test_accuracy = compute_accuracy(preds,labels)
            print('Epoch:',epoch, ';train and test accuracies:',train_accuracy,test_accuracy)

#     clear_output()
    plt.plot(train_iterations, train_losses, 'b',test_iterations, test_losses, 'r')
    plt.xlabel('Epoch')
    plt.ylabel('Loss') 
    plt.legend(['Train','Test'])
    plt.title('Epoch vs Loss')
#     plt.show()
    plt.savefig("./cls_losses.png") # save graph for training visualization

In [ ]:
# ###save stuff
# folder = 'results/'
# torch.save(net.state_dict(), folder+'classification_model')
# filehandler = open(folder+'classification_train_iterations', 'wb') 
# pickle.dump(train_iterations, filehandler)
# filehandler = open(folder+'classification_train_losses', 'wb') 
# pickle.dump(train_losses, filehandler)
# filehandler = open(folder+'classification_test_iterations', 'wb') 
# pickle.dump(test_iterations, filehandler)
# filehandler = open(folder+'classification_test_losses', 'wb') 
# pickle.dump(test_losses, filehandler)

In [ ]:
##plot training loss
import pickle


filehandler = open(folder+'classification_train_iterations', 'rb') 
train_iterations = pickle.load(filehandler)
filehandler = open(folder+'classification_train_losses', 'rb') 
train_losses = pickle.load(filehandler)
filehandler = open(folder+'classification_test_iterations', 'rb') 
test_iterations = pickle.load(filehandler)
filehandler = open(folder+'classification_test_losses', 'rb') 
test_losses = pickle.load(filehandler)

import os,math,numpy as np
import matplotlib
import matplotlib.pyplot as plt
import scipy.interpolate

matplotlib.rcParams['axes.linewidth'] = 5
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
plt.rc('font',size=35)

iter=train_iterations
fig=plt.figure()
ax=fig.add_subplot(1,1,1)
fig.set_size_inches(12*1.5,8*1.5)
xnew=np.linspace(min(iter),max(iter),200)  
test_loss_spline=scipy.interpolate.CubicSpline(iter,test_losses)
test_data = test_loss_spline(xnew)

ln=ax.plot(xnew,test_data,label="Test Loss",linewidth=5,color='#ff7f0e')
ax.tick_params(axis='y',colors='#ff7f0e')

ax2=ax.twinx()
xnew=np.linspace(min(iter),max(iter),200)  
train_loss_spline=scipy.interpolate.CubicSpline(iter,train_losses)
train_data = train_loss_spline(xnew)
ln+=ax2.plot(xnew,train_data,label="Train Loss",linewidth=5,color='#1f77b4')
ax2.tick_params(axis='y',colors='#1f77b4')
ax2.spines['right'].set_color('#1f77b4')
ax2.spines['left'].set_color('#ff7f0e')

labs=[l.get_label() for l in ln]
ax.legend(ln,labs,loc=0)

#ax.set_ylabel("Value")
ax.set_xlabel("#Iteration")
plt.savefig("Iteration.pdf",bbox_inches='tight',pad_inches=0)
plt.show()

In [ ]:
# compute accuracy on the entire test data

net = PointNetClassification(N_CLASSES).to(device)
net.load_state_dict(torch.load('results/classification_model'))
net.eval()

test_N = len(ModelNet40Data.test['pcds'])

class_total = np.zeros(N_CLASSES)
class_correct = np.zeros(N_CLASSES)
#entire test data too larget to load in one batch, do one by one instead
for i in range(test_N):
    pcds = np.zeros((1,1024,3))
    labels = np.zeros(1,dtype=int)
    pcds[0,:,:] = ModelNet40Data.test['pcds'][i]
    labels[0] = int(ModelNet40Data.test['labels'][i])

    data = torch.from_numpy(pcds).float().to(device)
    labels = torch.from_numpy(labels).to(device)

    preds,M2 = net(data)
    prediction = preds.max(1).indices.cpu().detach().numpy()[0]
    target = labels[0].cpu().detach().numpy()
    class_total[target] += 1
    if prediction == target:
        class_correct[target] += 1
print(class_total)
print(class_correct)

avg_class = 0.
for i in range(N_CLASSES):
    avg_class += class_correct[i]/class_total[i]
    
print('total_accuracy:',sum(class_correct)/sum(class_total))
print('average_class_accuracy:',avg_class/N_CLASSES)